# Neural Networks - Practical 2

After having trained and used your first neural network, you should be able to apply these skills to another data set. 

The dataset used in this exercise is from the UCI machine learning repository. It consists of measurements of fetal heart rate (FHR) and uterine contraction (UC) features on cardiotocograms classified by expert obstetricians.

A more detailed description can be found here: https://archive.ics.uci.edu/ml/datasets/Cardiotocography

We have already extracted the main data table and the csv file can be found in the data subdirectory. For reference, the original Excel file is also supplied. 

The task is to classify the dataset based on the measurements. Here, we use the setting having three classes (according to the NSP column):

Normal     = 1
Suspect    = 2
Pathologic = 3

Feel free to use the 10 class version using the CLASS column. 

Your task in this exercise is to load the data and train a simple neural network to either predict the three classes or later 10 classes. 

For training purposes it is acceptable to use a train-test split. 
(However, you might want to evaluate the performance using a 5-fold cross validation. As the presented approach uses the keras module of tensorflow, the GridSearch of sklearn cannot easiliy be applied)




## Required imports

Please note this practical also switched off some warnings. 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix



from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python import keras

from tensorflow.python.keras.models import Sequential


## Read in the data 

This file contains a not very biological dataset. It is comprised of customers and their shopping behavious. I chose this one, to indicate a bit of pre-processing. A task which will potentially be required by the task for next week. 

A more detailed introduction in data wrangling will be introduced in another lecture. 


In [3]:
df = pd.read_csv('./data/CTG.csv')
# drop unused information
df = df.drop(['b', 'e', 'Unnamed: 9', 'Unnamed: 31','Unnamed: 42','Unnamed: 44','A','B','C','D','E','AD','DE','LD','FS','SUSP'],axis=1)


In [4]:
df.describe()

,AC,FM,UC,DL,DS,DP,DR,LB,AC.1,FM.1,...,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,CLASS,NSP
count,2126.000000,2127.000000,2127.000000,2128.000000,2128.000000,2128.000000,2128.0,2126.000000,2126.000000,2127.000000,...,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000
mean,2.722484,7.503056,3.669017,1.576128,0.003759,0.127820,0.0,133.303857,0.003178,0.009702,...,164.025400,4.068203,0.323612,137.452023,134.610536,138.090310,18.808090,0.320320,4.509878,1.304327
std,3.560850,39.030452,2.877148,2.517794,0.061213,0.471687,0.0,9.840844,0.003866,0.047762,...,17.944183,2.949386,0.706059,16.381289,15.593596,14.466589,28.977636,0.610829,3.026883,0.614377
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,106.000000,0.000000,0.000000,...,122.000000,0.000000,0.000000,60.000000,73.000000,77.000000,0.000000,-1.000000,1.000000,1.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,126.000000,0.000000,0.000000,...,152.000000,2.000000,0.000000,129.000000,125.000000,129.000000,2.000000,0.000000,2.000000,1.000000
50%,1.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.0,133.000000,0.002000,0.000000,...,162.000000,3.000000,0.000000,139.000000,136.000000,139.000000,7.000000,0.000000,4.000000,1.000000
75%,4.000000,2.000000,5.000000,3.000000,0.000000,0.000000,0.0,140.000000,0.006000,0.003000,...,174.000000,6.000000,0.000000,148.000000,145.000000,148.000000,24.000000,1.000000,7.000000,1.000000
max,26.000000,564.000000,23.000000,16.000000,1.000000,4.000000,0.0,160.000000,0.019000,0.481000,...,238.000000,18.000000,10.000000,187.000000,182.000000,186.000000,269.000000,1.000000,10.000000,3.000000


In [5]:
df.columns.values


array(['AC', 'FM', 'UC', 'DL', 'DS', 'DP', 'DR', 'LB', 'AC.1', 'FM.1',
       'UC.1', 'DL.1', 'DS.1', 'DP.1', 'ASTV', 'MSTV', 'ALTV', 'MLTV',
       'Width', 'Min', 'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean', 'Median',
       'Variance', 'Tendency', 'CLASS', 'NSP'], dtype=object)

## Remove the CLASS attribute

The column CLASS contains more detailed classification, when compared to NSP. Hence, we do not want to use it for learning and the column is removed. The results is saved in a new dataframe called df_new.

In [6]:
df_new = df.drop(['CLASS'],axis=1)
df_new = df.dropna()


## On your own

From here on, please use the skills you have learned so far to:

1. Split the data into X and y
2. Split the result into training and test (or even a 5-fold cross validation)
3. Apply scaling for numerical variables and an appropriate encoding for cetegorical ones
4. Set up a (multi-)layer neural network
5. Train the network and report on its performance.


1. Split the data into X and y

In [7]:
X = df_new[['AC', 'FM', 'UC', 'DL', 'DS', 'DP', 'DR', 'LB', 'AC.1', 'FM.1', 'UC.1', 'DL.1', 'DS.1', 'DP.1', 'ASTV', 'MSTV', 'ALTV', 'MLTV', 'Width', 'Min', 'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean', 'Median', 'Variance', 'Tendency']]
y = df_new['NSP']



2. Split the result into training and test (or even a 5-fold cross validation)



In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

3. Apply scaling for numerical variables and an appropriate encoding for cetegorical ones



In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
onehotencoder_labels = OneHotEncoder()

onehotencoder_labels.fit(np.array([y_train]).transpose()) 

# ecode using the new representation
y2_train = onehotencoder_labels.transform(np.array(np.array([y_train]).transpose())).toarray()
y2_test = onehotencoder_labels.transform(np.array(np.array([y_test]).transpose())).toarray()

4. Set up a (multi-)layer neural network



In [11]:
#Initializing Neural Network
neural_network = Sequential()
neural_network.add(Dense(activation = 'relu', input_dim = 28, units=10))
neural_network.add(Dense(activation = 'relu', units=10))
neural_network.add(Dense(activation = 'sigmoid', units=6))
neural_network.add(Dense(activation = 'softmax', units=3))


Instructions for updating:
Colocations handled automatically by placer.


5. Train the network and report on its performance.



In [12]:
neural_network.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
neural_network.fit(X_train, y2_train, batch_size = 10, nb_epoch = 10)
y_pred = neural_network.predict(X_test)




Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1488/1488 [==============================] - 0s 267us/sample - loss: 0.5452 - acc: 0.7392
Epoch 2/10
1488/1488 [==============================] - 0s 110us/sample - loss: 0.3585 - acc: 0.8537
Epoch 3/10
1488/1488 [==============================] - 0s 117us/sample - loss: 0.2847 - acc: 0.8860
Epoch 4/10
1488/1488 [==============================] - 0s 129us/sample - loss: 0.2511 - acc: 0.8920
Epoch 5/10
1488/1488 [==============================] - 0s 124us/sample - loss: 0.2349 - acc: 0.8911
Epoch 6/10
1488/1488 [==============================] - 0s 116us/sample - loss: 0.2234 - acc: 0.8976
Epoch 7/10
1488/1488 [==============================] - 0s 114us/sample - loss: 0.2143 - acc: 0.9034
Epoch 8/10
1488/1488 [==============================] - 0s 116us/sample - loss: 0.2062 - acc: 0.9108
Epoch 9/10
1488/1488 [==============================] - 0s 146us/sample - loss: 0.1995 - acc: 0.9111
Epoch 10/10
1488/1488 [====================

You can use the reverse of the OneHotEncoder to map back into numerical classes.

In [13]:
y_test_pred = onehotencoder_labels.inverse_transform(y_pred)
confusion_matrix(y_test,y_test_pred)

array([[486,  26,   1],
       [ 23,  56,   0],
       [  5,  17,  24]])

You can also create a report on the prediction:

In [14]:
from sklearn.metrics import classification_report

target_names = ['Normal','Suspect','Pathologic']
print(classification_report(y_test,y_test_pred,target_names=target_names))

              precision    recall  f1-score   support

      Normal       0.95      0.95      0.95       513
     Suspect       0.57      0.71      0.63        79
  Pathologic       0.96      0.52      0.68        46

    accuracy                           0.89       638
   macro avg       0.82      0.73      0.75       638
weighted avg       0.90      0.89      0.89       638

